In [1]:
from dotenv import load_dotenv
import os
load_dotenv("/media/uberdev/ddrv/gitFolders/codeai_fusion/.env")
from dspy import OpenAI

/home/uberdev/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from openinference.instrumentation.dspy import DSPyInstrumentor
# instruments the internal calls in DSPy library
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
# help to get the span of http requests to the APIs
from opentelemetry.sdk import trace as trace_sdk
#processes the data collected from the spans
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.semconv.resource import ResourceAttributes
import phoenix as px

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix
INFO:phoenix.inferences.inferences:Dataset: phoenix_inferences_f772533b-cfd7-474a-aebb-3a9eb89c3a45 initialized


In [3]:
session = px.launch_app()

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
Average Metric: 0.0 / 4  (0.0):  80%|████████  | 4/5 [34:29<08:37, 517.39s/it]


In [4]:
endpoint = "http://127.0.0.1:6006/v1/traces"
client = px.Client(endpoint=endpoint)
# https://docs.arize.com/phoenix/tracing/how-to-tracing/trace-a-deployed-app
resource = Resource(attributes={
    ResourceAttributes.PROJECT_NAME: 'instrumenting-dspy'
})

In [6]:
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)

DSPyInstrumentor().instrument(skip_dep_check=True) # here where DSPy is instrumented

In [7]:
# Need to check if the dspy is writing logs
import os
llm = OpenAI(model='gpt-4o-mini',
             api_key=os.environ['OPENAI_API_KEY'],
             max_tokens=2000)
llm("testing the traces in the server")

["It seems like you're looking to test traces in a server environment. Here are some general steps you can follow to test traces effectively:\n\n1. **Identify the Tracing Tool**: Determine which tracing tool or framework you are using (e.g., OpenTelemetry, Jaeger, Zipkin, etc.).\n\n2. **Set Up the Environment**: Ensure that your server environment is properly set up to collect traces. This may involve configuring your application to send trace data to the tracing backend.\n\n3. **Instrument Your Code**: Add tracing instrumentation to your application code. This typically involves adding specific libraries or SDKs and using them to create spans around key operations.\n\n4. **Generate Traffic**: Simulate user activity or generate load on your application to create traces. This can be done through automated testing tools or by manually interacting with the application.\n\n5. **Check Trace Collection**: Verify that traces are being collected by your tracing backend. You can do this by chec

In [17]:
dspy.settings.configure(lm=llm)

In [62]:
news

'Turkish hunger striker released for remainder of trial'

In [63]:
import dspy
class NewsCategorization(dspy.Signature):
    news_body = dspy.InputField(desc="The body of the news to be categorized")
    answer = dspy.OutputField(desc="Should be 'fake' or 'real'")

class CoTCombined(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(NewsCategorization)
        self.history = []  # This will store the history of operations

    def forward(self, news_body):
        # planning to making multiple predictions later
        # pred_one = self.prog(news_body=news) # << The variable news was wrongly assigned
        pred_one = self.prog(news_body=news_body)
        return dspy.Prediction(answer=pred_one)

In [64]:
base_cot = CoTCombined()

In [14]:
import pandas as pd
custom_trainset = []
custom_devset = []

with open('train_fake_real_news.tsv', 'r') as tsv:
    lines = tsv.readlines()
    for line in lines[1:21]:
        news, truth = line.split("\t")
        if truth.strip() == '0':
            custom_trainset.append(dspy.Example(news_body=news, answer='fake').with_inputs("news_body"))
        else:
            custom_trainset.append(dspy.Example(news_body=news, answer='real').with_inputs("news_body"))
    for line in lines[22:43]:
        news, truth = line.split("\t")
        if truth.strip() == '0':
            custom_devset.append(dspy.Example(news_body=news, answer='fake').with_inputs("news_body"))
        else:
            custom_devset.append(dspy.Example(news_body=news, answer='real').with_inputs("news_body"))

In [15]:
custom_devset[0]

Example({'news_body': 'TRUMP HITS BACK After Cowgirl Congresswoman Trashes Him Over Words Said to Grieving Widow', 'answer': 'fake'}) (input_keys={'news_body'})

In [31]:
base_cot(custom_devset[0]).answer.answer

'real'

In [21]:
def validate_answer(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

In [23]:
# The metric below will return a float if trace is None (i.e., 
# if it's used for evaluation or optimization), 
# means there will no trace during evaluation or optimization
 
# Trace is generated if metric is used to bootstrap demonstrations.

In [ ]:
# bootstrapping, i.e. self-generating good demonstrations of each step

In [59]:
def validate_trace_n_answer(example, pred, trace=[]):
    # check the gold label and the predicted answer are the same
    answer_match = example.answer.lower() == pred.answer.answer.lower()
    # print(f"Trace is {trace}")
    if len(trace) > 0:
        print(f"Trace is {trace}")
    else:
        print("There is no Trace")
    return answer_match

In [40]:
# lets evaluate couple of datapoints
from dspy.evaluate import Evaluate

In [65]:
evaluate = Evaluate(devset=custom_devset[:5], display_progress=True, display_table=True)

In [66]:
# send the CoT program into the evaluate
evaluation_var = evaluate(program=base_cot, metric=validate_trace_n_answer)

Average Metric: 5 / 5  (100.0): 100%|██████████| 5/5 [00:00<00:00, 13.66it/s]


,news_body,example_answer,pred_answer,validate_trace_n_answer
0,TRUMP HITS BACK After Cowgirl Congresswoman Trashes Him Over Words Said to Grieving Widow,fake,"Prediction( rationale='determine the authenticity of this news. The headline mentions a well-known political figure, Donald Trump, and a specific incident involving a congresswoman. The phrasing...",✔️ [True]
1,Heroin Addict Trump Voter Sad Now That Donald Is Taking Away His Treatments (VIDEO),fake,Prediction( rationale='determine the credibility of this news. The title suggests a sensational story that combines political commentary with a personal narrative about addiction. The use...,✔️ [True]
2,Republican Compares Lincoln To Hitler; This Time It Wasn’t Sean Spicer,fake,"Prediction( rationale='determine the credibility of this news. The headline suggests a controversial comparison made by a Republican figure, which could be sensationalized or taken out...",✔️ [True]
3,"NBC Gets Brutal With Donald Trump, Releasing The Video He Never Wanted Us To See (VIDEO)",fake,Prediction( rationale='determine the authenticity of this news. The title suggests that NBC has released a video that Donald Trump did not want to be made...,✔️ [True]
4,THIS BIG ANNOUNCEMENT IS YET ANOTHER REASON NOT TO VOTE FOR HILLARY CLINTON,fake,Prediction( rationale='determine the credibility of this news. The statement appears to be politically charged and lacks specific evidence or sources to support its claims. It...,✔️ [True]


In [43]:
# what does dspy.optimizers tune?
# DSPy programs consist of multiple calls to LMs, stacked together as [DSPy modules]. Each DSPy module has internal 
# parameters of three kinds: 
# (1) the LM weights,
# (2) the instructions, 
# and (3) demonstrations of the input/output behavior.

In [44]:
# Given a metric, DSPy can optimize all of these three with multi-stage optimization algorithms. 
# These can combine gradient descent (for LM weights) and discrete LM-driven optimization, 
# i.e. for crafting/updating instructions and for creating/validating demonstrations. 
# DSPy Demonstrations are like few-shot examples, but they're far more powerful.
# They can be created from scratch, given your program, and their creation and 
# selection can be optimized in many effective ways

In [46]:
#let call a basic bootstrap optimizer
from dspy.teleprompt import BootstrapFewShot

**BootstrapFewShot:** 

Uses a teacher module (which defaults to your program) to generate complete demonstrations for every stage of your program, along with labeled examples in trainset. 

Parameters include max_labeled_demos (the number of demonstrations randomly selected from the trainset) and max_bootstrapped_demos (the number of additional examples generated by the teacher).

The bootstrapping process employs the metric to validate demonstrations, including only those that pass the metric in the "compiled" prompt. 

Advanced: Supports using a teacher program that is a different DSPy program that has compatible structure, for harder tasks.

In [67]:
config = dict(max_bootstrapped_demos=4,
              max_labeled_demos=4,
             max_rounds=3)

teleprompter = BootstrapFewShot(metric=validate_trace_n_answer, **config)

In [49]:
len(custom_trainset)

20

In [55]:
custom_trainset

[Example({'news_body': ' Courts Decide Conspiracy Nut Alex Jones Is Too Crazy To Raise His Own Kids (DETAILS)', 'answer': 'fake'}) (input_keys={'news_body'}),
 Example({'news_body': "U.S. Senator Menendez's corruption trial to proceed: judge", 'answer': 'real'}) (input_keys={'news_body'}),
 Example({'news_body': 'Search ends for bodies in Mexico City after earthquake', 'answer': 'real'}) (input_keys={'news_body'}),
 Example({'news_body': 'BUH-BYE! GLENN BECK Places Final Nail In His Coffin…And His Former Fans Won’t Miss Him [VIDEO]', 'answer': 'fake'}) (input_keys={'news_body'}),
 Example({'news_body': 'BREAKING: Michigan Native KID ROCK Announces He’s Running For US Senate', 'answer': 'fake'}) (input_keys={'news_body'}),
 Example({'news_body': 'Medicaid cuts coming in Trump budget: Washington Post', 'answer': 'real'}) (input_keys={'news_body'}),
 Example({'news_body': 'GOP MAJORITY SENATE FINALLY GETS IT RIGHT: Votes To Gut Obamacare And Defund Planned Parenthood', 'answer': 'fake'}) 

In [68]:
optimized_program = teleprompter.compile(base_cot, trainset=custom_trainset)

 10%|█         | 2/20 [00:00<00:02,  7.19it/s]

Trace is [(Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)), {'news_body': ' Courts Decide Conspiracy Nut Alex Jones Is Too Crazy To Raise His Own Kids (DETAILS)'}, Prediction(
    rationale='produce the answer. We analyze the news body which discusses a legal decision regarding Alex Jones, a controversial figure known for promoting conspiracy theories. The phrasing "too cr

 20%|██        | 4/20 [00:02<00:13,  1.20it/s]

Trace is [(Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)), {'news_body': 'BUH-BYE! GLENN BECK Places Final Nail In His Coffin…And His Former Fans Won’t Miss Him [VIDEO]'}, Prediction(
    rationale='determine the credibility of this news. The title suggests a sensationalist approach, using phrases like "final nail in his coffin" which indicates a dramatic or exaggerated t

 25%|██▌       | 5/20 [00:04<00:14,  1.03it/s]


Trace is [(Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)), {'news_body': 'BREAKING: Michigan Native KID ROCK Announces He’s Running For US Senate'}, Prediction(
    rationale='determine the credibility of this news. The news mentions a well-known public figure, Kid Rock, announcing a political run, which is a significant event that could be reported by multiple credible s

  0%|          | 0/20 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 2.


In [69]:
optimized_program.dump_state()

[('prog', Predict(StringSignature(news_body -> rationale, answer
    instructions='Given the fields `news_body`, produce the fields `answer`.'
    news_body = Field(annotation=str required=True json_schema_extra={'desc': 'The body of the news to be categorized', '__dspy_field_type': 'input', 'prefix': 'News Body:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': "Should be 'fake' or 'real'", '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)))]


{'prog': {'lm': None,
  'traces': [],
  'train': [],
  'demos': [Example({'augmented': True, 'news_body': "U.S. Senator Menendez's corruption trial to proceed: judge", 'rationale': 'determine the credibility of this news. The news mentions a specific U.S. Senator, Bob Menendez, and refers to a legal proceeding, which is a verifiable event. The source of the information is a judge, which adds to its reliability. There are no indications of sensationalism or misinformation in the statement. Therefore, it is reasonable to conclude that this news is credible.', 'answer': 'real'}) (input_keys=None),
   Example({'augmented': True, 'news_body': 'Search ends for bodies in Mexico City after earthquake', 'rationale': 'determine the authenticity of this news. The news mentions a search for bodies following an earthquake, which is a plausible and serious event that could occur in a city like Mexico City, known for its seismic activity. The source, Anadolu, is a reputable news agency. Given these f